In [138]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [139]:
from glob import glob 
from fastcore.xtras import load_pickle

import pandas as pd
from gptchem.data import get_photoswitch_data

In [140]:
data = get_photoswitch_data()
data_subset = data.dropna(
    subset=["E isomer pi-pi* wavelength in nm", "SMILES", "Z isomer pi-pi* wavelength in nm"]
)
subset_smiles = data_subset["SMILES"].tolist()

In [141]:
def get_prevalence(smiles, fragment):
    return sum([fragment in x for x in smiles]) / len(smiles)


In [142]:
all_res = glob('out/*/*.pkl')

In [143]:
all_res

['out/20230207_033214/summary.pkl',
 'out/20230207_134940/summary.pkl',
 'out/20230207_100850/summary.pkl',
 'out/20230206_131524/summary.pkl',
 'out/20230207_121519/summary.pkl',
 'out/20230207_050012/summary.pkl',
 'out/20230207_134423/summary.pkl',
 'out/20230207_122451/summary.pkl',
 'out/20230207_044336/summary.pkl',
 'out/20230207_143050/summary.pkl',
 'out/20230207_145300/summary.pkl',
 'out/20230207_091227/summary.pkl',
 'out/20230206_115656/summary.pkl',
 'out/20230207_111718/summary.pkl',
 'out/20230206_211350/summary.pkl',
 'out/20230207_123048/summary.pkl',
 'out/20230207_112940/summary.pkl',
 'out/20230206_090056/summary.pkl',
 'out/20230206_211450/summary.pkl',
 'out/20230207_164223/summary.pkl']

In [144]:
compiled = []

for res in all_res:
    summary = load_pickle(res)
    for temp_res in summary['res_at_temp']:
        compiled.append({
            'train_size': summary['train_size'],
            'noise_level': summary['noise_level'],
            'num_samples': summary['num_samples'],
            'temperature': temp_res['temperature'],
            'valid_smiles': temp_res['valid_smiles'],
            'novel_smiles': temp_res['novel_smiles'],
            'valid_indices': temp_res['valid_indices'],
            'novel_indices': temp_res['novel_indices'],
            'frac_group': temp_res['fragment_fraction'],
            'constrain_satisfaction': temp_res['constrain_satisfaction'],
            'constrain_satisfaction_novel': temp_res['constrain_satisfaction_novel'],
            'prevalence': get_prevalence(temp_res['train_smiles'], summary['group']),
            'group': summary['group'],
        })

In [145]:
compiled = pd.DataFrame(compiled)

In [146]:
compiled['enrichment'] = compiled['frac_group'] / compiled['prevalence']

In [147]:
comp = compiled.groupby(['temperature', 'group', 'noise_level']).agg(['mean', 'std'])

/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ipykernel_59489/1644170794.py:1: FutureWarning: ['valid_smiles', 'novel_smiles', 'valid_indices', 'novel_indices', 'constrain_satisfaction', 'constrain_satisfaction_novel'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  comp = compiled.groupby(['temperature', 'group', 'noise_level']).agg(['mean', 'std'])


In [148]:
comp.loc[1.0]

train_size      num_samples             frac_group  \
                        mean  std        mean         std       mean   
group noise_level                                                      
Br    0.5               92.0  0.0       300.0    0.000000   0.094851   
C#C   0.5               92.0  NaN       300.0         NaN   0.000000   
      50.0              92.0  NaN       300.0         NaN   0.000000   
C#CBr 0.5               92.0  0.0       300.0    0.000000   0.000000   
      50.0              92.0  NaN       300.0         NaN   0.000000   
C#CC  0.5               92.0  0.0       300.0    0.000000   0.007692   
      50.0              92.0  NaN       300.0         NaN   0.015625   
C=O   0.5               92.0  NaN       300.0         NaN   0.016129   
      50.0              92.0  NaN       300.0         NaN   0.000000   
Cl    0.5               92.0  NaN       300.0         NaN   0.074627   
      50.0              92.0  NaN       300.0         NaN   0.079365   
F     0.1               92.0  NaN       300.0         NaN   0.271429   
      0.5               92.0  NaN       300.0         NaN   0.378788   
      50.0              92.0  NaN       300.0         NaN   0.246575   
I     0.5               92.0  0.0       200.0  141.421356   0.007692   

                            prevalence      enrichment            
                        std       mean  std       mean       std  
group noise_level                                                 
Br    0.5          0.022662   0.065217  0.0   1.454388  0.347482  
C#C   0.5               NaN   0.000000  NaN        NaN       NaN  
      50.0              NaN   0.000000  NaN        NaN       NaN  
C#CBr 0.5          0.000000   0.000000  0.0        NaN       NaN  
      50.0              NaN   0.000000  NaN        NaN       NaN  
C#CC  0.5          0.010879   0.000000  0.0        inf       NaN  
      50.0              NaN   0.000000  NaN        inf       NaN  
C=O   0.5               NaN   0.000000  NaN        inf       NaN  
      50.0              NaN   0.000000  NaN        NaN       NaN  
Cl    0.5               NaN   0.097826  NaN   0.762852       NaN  
      50.0              NaN   0.097826  NaN   0.811287       NaN  
F     0.1               NaN   0.304348  NaN   0.891837       NaN  
      0.5               NaN   0.304348  NaN   1.244589       NaN  
      50.0              NaN   0.304348  NaN   0.810176       NaN  
I     0.5          0.010879   0.010870  0.0   0.707692  1.000828

In [149]:
comp.loc[0.5]

train_size      num_samples             frac_group  \
                        mean  std        mean         std       mean   
group noise_level                                                      
Br    0.5               92.0  0.0       300.0    0.000000   0.088512   
C#C   0.5               92.0  NaN       300.0         NaN   0.000000   
      50.0              92.0  NaN       300.0         NaN   0.000000   
C#CBr 0.5               92.0  0.0       300.0    0.000000   0.000000   
      50.0              92.0  NaN       300.0         NaN   0.000000   
C#CC  0.5               92.0  0.0       300.0    0.000000   0.000000   
      50.0              92.0  NaN       300.0         NaN   0.000000   
C=O   0.5               92.0  NaN       300.0         NaN   0.000000   
      50.0              92.0  NaN       300.0         NaN   0.000000   
Cl    0.5               92.0  NaN       300.0         NaN   0.155556   
      50.0              92.0  NaN       300.0         NaN   0.119565   
F     0.1               92.0  NaN       300.0         NaN   0.329545   
      0.5               92.0  NaN       300.0         NaN   0.450549   
      50.0              92.0  NaN       300.0         NaN   0.244444   
I     0.5               92.0  0.0       200.0  141.421356   0.000000   

                            prevalence      enrichment            
                        std       mean  std       mean       std  
group noise_level                                                 
Br    0.5          0.038236   0.065217  0.0   1.357183  0.586292  
C#C   0.5               NaN   0.000000  NaN        NaN       NaN  
      50.0              NaN   0.000000  NaN        NaN       NaN  
C#CBr 0.5          0.000000   0.000000  0.0        NaN       NaN  
      50.0              NaN   0.000000  NaN        NaN       NaN  
C#CC  0.5          0.000000   0.000000  0.0        NaN       NaN  
      50.0              NaN   0.000000  NaN        NaN       NaN  
C=O   0.5               NaN   0.000000  NaN        NaN       NaN  
      50.0              NaN   0.000000  NaN        NaN       NaN  
Cl    0.5               NaN   0.097826  NaN   1.590123       NaN  
      50.0              NaN   0.097826  NaN   1.222222       NaN  
F     0.1               NaN   0.304348  NaN   1.082792       NaN  
      0.5               NaN   0.304348  NaN   1.480377       NaN  
      50.0              NaN   0.304348  NaN   0.803175       NaN  
I     0.5          0.000000   0.010870  0.0   0.000000  0.000000

In [150]:
comp.loc[2.0]

train_size      num_samples             frac_group  \
                        mean  std        mean         std       mean   
group noise_level                                                      
Br    0.5               92.0  0.0       300.0    0.000000   0.000000   
C#C   0.5               92.0  NaN       300.0         NaN   0.000000   
      50.0              92.0  NaN       300.0         NaN   0.000000   
C#CBr 0.5               92.0  0.0       300.0    0.000000   0.000000   
      50.0              92.0  NaN       300.0         NaN        NaN   
C#CC  0.5               92.0  0.0       300.0    0.000000   0.000000   
      50.0              92.0  NaN       300.0         NaN   0.000000   
C=O   0.5               92.0  NaN       300.0         NaN   0.000000   
      50.0              92.0  NaN       300.0         NaN   0.000000   
Cl    0.5               92.0  NaN       300.0         NaN   0.066667   
      50.0              92.0  NaN       300.0         NaN        NaN   
F     0.1               92.0  NaN       300.0         NaN   0.333333   
      0.5               92.0  NaN       300.0         NaN   0.200000   
      50.0              92.0  NaN       300.0         NaN   0.125000   
I     0.5               92.0  0.0       200.0  141.421356   0.041667   

                            prevalence      enrichment            
                        std       mean  std       mean       std  
group noise_level                                                 
Br    0.5          0.000000   0.065217  0.0   0.000000  0.000000  
C#C   0.5               NaN   0.000000  NaN        NaN       NaN  
      50.0              NaN   0.000000  NaN        NaN       NaN  
C#CBr 0.5          0.000000   0.000000  0.0        NaN       NaN  
      50.0              NaN   0.000000  NaN        NaN       NaN  
C#CC  0.5               NaN   0.000000  0.0        NaN       NaN  
      50.0              NaN   0.000000  NaN        NaN       NaN  
C=O   0.5               NaN   0.000000  NaN        NaN       NaN  
      50.0              NaN   0.000000  NaN        NaN       NaN  
Cl    0.5               NaN   0.097826  NaN   0.681481       NaN  
      50.0              NaN   0.097826  NaN        NaN       NaN  
F     0.1               NaN   0.304348  NaN   1.095238       NaN  
      0.5               NaN   0.304348  NaN   0.657143       NaN  
      50.0              NaN   0.304348  NaN   0.410714       NaN  
I     0.5          0.058926   0.010870  0.0   3.833333  5.421152

In [151]:
comp.loc[0.75]

train_size      num_samples             frac_group  \
                        mean  std        mean         std       mean   
group noise_level                                                      
Br    0.5               92.0  0.0       300.0    0.000000   0.112260   
C#C   0.5               92.0  NaN       300.0         NaN   0.000000   
      50.0              92.0  NaN       300.0         NaN   0.000000   
C#CBr 0.5               92.0  0.0       300.0    0.000000   0.000000   
      50.0              92.0  NaN       300.0         NaN   0.000000   
C#CC  0.5               92.0  0.0       300.0    0.000000   0.005882   
      50.0              92.0  NaN       300.0         NaN   0.000000   
C=O   0.5               92.0  NaN       300.0         NaN   0.000000   
      50.0              92.0  NaN       300.0         NaN   0.000000   
Cl    0.5               92.0  NaN       300.0         NaN   0.137500   
      50.0              92.0  NaN       300.0         NaN   0.097561   
F     0.1               92.0  NaN       300.0         NaN   0.310345   
      0.5               92.0  NaN       300.0         NaN   0.400000   
      50.0              92.0  NaN       300.0         NaN   0.311111   
I     0.5               92.0  0.0       200.0  141.421356   0.000000   

                            prevalence      enrichment            
                        std       mean  std       mean       std  
group noise_level                                                 
Br    0.5          0.016378   0.065217  0.0   1.721327  0.251127  
C#C   0.5               NaN   0.000000  NaN        NaN       NaN  
      50.0              NaN   0.000000  NaN        NaN       NaN  
C#CBr 0.5          0.000000   0.000000  0.0        NaN       NaN  
      50.0              NaN   0.000000  NaN        NaN       NaN  
C#CC  0.5          0.008319   0.000000  0.0        inf       NaN  
      50.0              NaN   0.000000  NaN        NaN       NaN  
C=O   0.5               NaN   0.000000  NaN        NaN       NaN  
      50.0              NaN   0.000000  NaN        NaN       NaN  
Cl    0.5               NaN   0.097826  NaN   1.405556       NaN  
      50.0              NaN   0.097826  NaN   0.997290       NaN  
F     0.1               NaN   0.304348  NaN   1.019704       NaN  
      0.5               NaN   0.304348  NaN   1.314286       NaN  
      50.0              NaN   0.304348  NaN   1.022222       NaN  
I     0.5          0.000000   0.010870  0.0   0.000000  0.000000